### Step1. 训练集PNG转JPG，长边缩放至512

In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from glob import glob
import json
import numpy as np
from pathlib import Path
from tqdm import tqdm
from utils.table_helper import resize_image_long, resize_image_short



In [2]:
# 数据集名称
DATASET = "train"
DATASET_ROOT = "/media/ubuntu/Date12/TableStruct/data/"
DATASET_ROOT_NEW = "/media/ubuntu/Date12/TableStruct/new_data/"
# 输入数据目录
TRAIN_PNG_DIR  = os.path.join(DATASET_ROOT, DATASET)
# 输出
TRAIN_JPG_DIR  = os.path.join(DATASET_ROOT_NEW, f"{DATASET}_jpg480max")
JPG_JSON_PATH  = os.path.join(DATASET_ROOT_NEW, f"{DATASET}_jpg480max.json")

Path(TRAIN_JPG_DIR).mkdir(parents=True, exist_ok=True)



jpg_infos = dict()
for img_path in tqdm(sorted(glob(os.path.join(TRAIN_PNG_DIR, "*.png")))):
    img_file = os.path.basename(img_path)
    img_id = img_file.split(".")[0]
    jpg_img_path = os.path.join(TRAIN_JPG_DIR, img_file.replace(".png", ".jpg"))
    img = cv2.imread(img_path)

    oriwidth, oriheight = img.shape[1], img.shape[0]
    img = resize_image_long(img, long_side=480)
    width, height = img.shape[1], img.shape[0]
    scale = width / oriwidth

    jpg_infos[img_id] = dict(oriwidth=oriwidth, oriheight=oriheight, width=width, height=height, scale=scale)
    cv2.imwrite(jpg_img_path, img)

json.dump(jpg_infos, open(JPG_JSON_PATH, "w"), indent=4)


100%|██████████| 12104/12104 [38:50<00:00,  5.19it/s] 


### Step2. 训练集JSON坐标标注按比例缩放

In [3]:
def segmentation_scale(segmentations, scale):
    ret = []
    for seg in segmentations:
        seg = np.array(seg) * scale
        seg = seg.tolist()
        ret.append(seg)
    return ret


# 数据集名称
DATASET = "train"
DATASET_ROOT = "/media/ubuntu/Date12/TableStruct/data/"
DATASET_ROOT_NEW = "/media/ubuntu/Date12/TableStruct/new_data/"
# 输入数据目录
TRAIN_JSON_DIR  = os.path.join(DATASET_ROOT, DATASET)
# 输出
TRAIN_JPG_DIR  = os.path.join(DATASET_ROOT_NEW, f"{DATASET}_jpg480max")
JPG_JSON_PATH  = os.path.join(DATASET_ROOT_NEW, f"{DATASET}_jpg480max.json")
Path(TRAIN_JPG_DIR).mkdir(parents=True, exist_ok=True)


img_infos = json.load(open(JPG_JSON_PATH, "r"))
for json_path in tqdm(sorted(glob(os.path.join(TRAIN_JSON_DIR, "*.json")))):
    json_file = os.path.basename(json_path)
    img_id = json_file.split(".")[0]
    jpg_json_path = os.path.join(TRAIN_JPG_DIR, json_file)

    info = json.load(open(json_path, "r"))
    scale = img_infos[img_id]["scale"]

    info["row"]  = segmentation_scale(info["row"], scale)
    info["col"]  = segmentation_scale(info["col"], scale)
    info["line"] = segmentation_scale(info["line"], scale)
    info["cell"] = segmentation_scale(info["cell"], scale)

    json.dump(info, open(jpg_json_path, "w"))

100%|██████████| 12104/12104 [00:41<00:00, 291.57it/s]
